# Du modèle d'AGODA à un topic modelling adapté à mon corpus

## Conversion des pdf en txt

In [15]:
import os
from PyPDF2 import PdfReader

def extract_text_from_pdfs(root_folder, destination_folder):
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            if file.endswith(".pdf"):
                file_path = os.path.join(root, file)
                pdf = PdfReader(file_path)
                text = ""
                for page in range(len(pdf.pages)):
                    text += pdf.pages[page].extract_text()
                text_file_path = os.path.join(destination_folder, file.replace(".pdf", ".txt"))
                with open(text_file_path, "w", encoding="utf-8") as text_file:
                    text_file.write(text)

# Utilisez la fonction comme suit :
extract_text_from_pdfs("/Volumes/Elements/JO_débats_1956-1958/", "/Volumes/Elements/JO_débats_1956-1958/débats_texte/")

## Entraînement du modèle

## Exécution du code adapté de Marie Puren

In [4]:
pip install git+https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer.git

  Cloning https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer.git to /private/var/folders/py/k9l9n4nj5l966__zkxgrfsj00000gn/T/pip-req-build-rilpsybh
  Running command git clone --filter=blob:none --quiet https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer.git /private/var/folders/py/k9l9n4nj5l966__zkxgrfsj00000gn/T/pip-req-build-rilpsybh
  Resolved https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer.git to commit bc0ebd0135a6cc78f48ddf184069b4c0b9c017d8
  Preparing metadata (setup.py) ... done
  Created wheel for FrenchLefffLemmatizer: filename=FrenchLefffLemmatizer-0.3-py3-none-any.whl size=3533513 sha256=bed0659c54558f8b025743e34e6c16b3a589ce9515d9528266d7bc32517734a5
  Stored in directory: /private/var/folders/py/k9l9n4nj5l966__zkxgrfsj00000gn/T/pip-ephem-wheel-cache-oz47tamb/wheels/98/2a/68/fa037d302c091a60390195bc19da0a9eb835ff105dbee42c7d
Successfully built FrenchLefffLemmatizer
Note: you may need to restart the kernel to use updated packages.


Suit le code de Marie Puren, modifié pour mon corpus. Comme je n'ai pas de modèle entraîné, je ne récupère pas tous les résultats proposés. 

In [38]:
import os
import pickle  # librairie pour save des modèles de machine learning
import re
from typing import List, TextIO

import numpy as np
import pandas as pd
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer as FLF
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

lemmatizer = FLF()

path_data = "/Volumes/Elements/JO_débats_1956-1958/débats_texte/"
path_to_model = os.path.join(path_data, "model_ML")

stop_words = set(list(open(os.path.join(path_data, "french_stopwords.txt"), "r", encoding="utf-8").read().split("\n"))
                 + stopwords.words("french"))


def table_ocr_1956_1958():
        print("récupération des fichiers")
        fichiers = os.listdir(os.path.join(path_data))
        data = pd.DataFrame(columns=["date", "text"])
        for name in fichiers:
            print(name)
            if name.split("-")[0][:2] == '19':
                file: TextIO = open(os.path.join(path_data, name), encoding="utf-8")
                text: str = file.read()
                file.close()
                data = data.append({'date': name.split('.')[0], "text": text}, ignore_index=True)
        return data

        
def build_corpus(df: pd.DataFrame):
    print("nettoyage du texte")
    data: List = []
    for i in range(df.shape[0]):
        print(f"{i}/{df.shape[0]}")
        text = " ".join(re.findall("[A-Za-zâêûîôäëüïöùàçéèÉ\-\.]+", df.text[i]))
        text = re.sub("([a-z])- ", r"\1", text)
        text = re.sub("\-", " ", text)
        text = re.sub("[M]+\. ([A-Z]+[a-zâêûîôäëüïöùàçéè]+(?:\s[A-Z]+[a-zâêûîôäëüïöùàçéè]+)?)", " ", text)
        text = re.sub("\.", " ", text)
        bag_of_words: List[str] = word_tokenize(text.lower(), language="french")
        bag_of_words = [w for w in bag_of_words if 1 <= len(w) < 22]
        bag_of_words = [lemmatizer.lemmatize(w).lower() for w in bag_of_words if w not in stop_words]
        data.append(bag_of_words)
    df.loc[:, "bag_of_words"] = data
    return df


def count_vectorizer(df: pd.DataFrame, p: int):
    print("compte des coccurences")
    data = [" ".join(w) for w in df.bag_of_words]
    vectorizer = CountVectorizer(max_features=p)
    X = vectorizer.fit_transform(data)
    word_frequency_matrix = pd.DataFrame(data=X.toarray(), index=df.date, columns=vectorizer.get_feature_names())
    word_frequency_matrix = word_frequency_matrix.sort_index()
    word_frequency_matrix.to_csv(os.path.join(path_data, "word_frequency_56-58.csv"),
                                 sep=";", encoding="utf-8", index=False)
    return word_frequency_matrix


def build_model(model_file):
    nb_topics: int = 50
    words_per_topic: int = 20
    if os.path.exists(os.path.join(path_to_model, model_file)):
        print("il existe déjà un modèle avec ce nom")
    else:
        word_frequency_matrix = pd.read_csv(os.path.join(path_data, "word_frequency_56-58.csv"), sep=";", encoding="utf-8",
                                            index_col=0)
        clefs: List[str] = list(word_frequency_matrix.columns)
        print(clefs[:10])
        blocs: List[str] = list(word_frequency_matrix.index)
        print(blocs[:10])
        print("Topic modeling")
        lda = LatentDirichletAllocation(n_components=nb_topics)
        topic_to_text = lda.fit_transform(word_frequency_matrix.values)
        pkl_filename = os.path.join(path_to_model, model_file)
        with open(pkl_filename, 'wb') as file:
            pickle.dump(lda, file)
    all_topics: pd.DataFrame = pd.DataFrame({f"Topic{i}": [clefs[w] for w in top.argsort()[-words_per_topic:]]
                                             for i, top in enumerate(lda.components_)})
    table_topics_to_texts: pd.DataFrame = pd.DataFrame(np.vectorize(lambda z: f"{z:.3f}")(topic_to_text),
                                                       columns=range(nb_topics), index=blocs)
    all_topics.to_excel(os.path.join(path_data, "topics.xlsx"), encoding="utf-8", index=False)
    table_topics_to_texts.to_excel(os.path.join(path_data, "corpus_topics.xlsx"), encoding="utf-8", index=True)


def load_model(model_file):
    word_freq = pd.read_csv(os.path.join(path_data, "word_frequency_56-58.csv"), sep=";", encoding="utf-8", index_col=0)
    clefs: List[str] = list(word_freq.columns)
    blocs: List[str] = list(word_freq.index)
    pkl_filename = os.path.join(path_to_model, model_file)
    with open(pkl_filename, 'rb') as file:
        lda = pickle.load(file)
    nb_topics: int = lda.n_components
    words_per_topic: int = 20
    topic_to_text = lda.transform(word_freq.values)
    all_topics: pd.DataFrame = pd.DataFrame({f"Topic{i}": [clefs[w] for w in top.argsort()[-words_per_topic:]]
                                             for i, top in enumerate(lda.components_)})
    table_topics_to_texts: pd.DataFrame = pd.DataFrame(np.vectorize(lambda z: f"{z:.3f}")(topic_to_text),
                                                       columns=range(nb_topics), index=blocs)
    all_topics.to_excel(os.path.join(path_data, "topics.xlsx"), encoding="utf-8", index=False)
    table_topics_to_texts.to_excel(os.path.join(path_data, "corpus_topics.xlsx"), encoding="utf-8", index=True)
    return topic_to_text, all_topics, table_topics_to_texts


def get_parameter(adr: str):
    pkl_filename = os.path.join(path_to_model, adr)
    with open(pkl_filename, 'rb') as file:
        lda: LatentDirichletAllocation = pickle.load(file)
    print(lda.get_params())


if __name__ == "__main__":

    df = table_ocr_1956_1958()  # collecte dans un dataframe, les textes des années 56-58 avec la date comme référence
    df = build_corpus(df)  # calcul les bag of words pour chaque document, puis les ajoute dans le dataframe précédent
    word_frequency = count_vectorizer(df, 10000)  # à partir des bag of words, calcul la matrice de fréquence par doc
    build_model('lda_model.pkl')  # lance un modèle de LDA. Il ne fera rien si un modèle du même nom existe déjà
    text_topics, topics, table_text_topics = load_model("lda_model.pkl")
    get_parameter("lda_model.pkl")

récupération des fichiers
.DS_Store
1956_i100.txt


/var/folders/py/k9l9n4nj5l966__zkxgrfsj00000gn/T/ipykernel_53439/2583446060.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'date': name.split('.')[0], "text": text}, ignore_index=True)
/var/folders/py/k9l9n4nj5l966__zkxgrfsj00000gn/T/ipykernel_53439/2583446060.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'date': name.split('.')[0], "text": text}, ignore_index=True)
/var/folders/py/k9l9n4nj5l966__zkxgrfsj00000gn/T/ipykernel_53439/2583446060.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'date': name.split('.')[0], "text": text}, ignore_index=True)
/var/folders/py/k9l9n4nj5l966__zkxgrfsj00000gn/T/ipykernel_53439/2583446060.py:33: FutureWarning: Th

1956_i101.txt
1956_i102.txt
1956_i103.txt
1956_i104.txt
1956_i105.txt
1956_i106.txt
1956_i107.txt
1956_i108.txt


/var/folders/py/k9l9n4nj5l966__zkxgrfsj00000gn/T/ipykernel_53439/2583446060.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'date': name.split('.')[0], "text": text}, ignore_index=True)
/var/folders/py/k9l9n4nj5l966__zkxgrfsj00000gn/T/ipykernel_53439/2583446060.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'date': name.split('.')[0], "text": text}, ignore_index=True)


1956_i109.txt
1956_i110.txt
1956_i111.txt


...

/var/folders/py/k9l9n4nj5l966__zkxgrfsj00000gn/T/ipykernel_53439/2583446060.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'date': name.split('.')[0], "text": text}, ignore_index=True)
/var/folders/py/k9l9n4nj5l966__zkxgrfsj00000gn/T/ipykernel_53439/2583446060.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'date': name.split('.')[0], "text": text}, ignore_index=True)
/var/folders/py/k9l9n4nj5l966__zkxgrfsj00000gn/T/ipykernel_53439/2583446060.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'date': name.split('.')[0], "text": text}, ignore_index=True)
/var/folders/py/k9l9n4nj5l966__zkxgrfsj00000gn/T/ipykernel_53439/2583446060.py:33: FutureWarning: Th

1958_i62.txt
1958_i7.txt
1958_i8.txt
1958_i9.txt
french_stopwords.txt
reponses_29_septembre_1956.txt
reponses_4_septembre_1956.txt
word_frequency_80.csv
nettoyage du texte
0/327
1/327
2/327
3/327
4/327
5/327
6/327
7/327
8/327
9/327
10/327
11/327
12/327
13/327
14/327
15/327
16/327
17/327
18/327
19/327
20/327
21/327
22/327
23/327
24/327
25/327
26/327
27/327
28/327
29/327
30/327
31/327
32/327
33/327
34/327
35/327
36/327
37/327
38/327
39/327
40/327
41/327
42/327
43/327
44/327
45/327
46/327
47/327
48/327
49/327
50/327
51/327
52/327
53/327
54/327
55/327
56/327
57/327
58/327
59/327
60/327
61/327
62/327
63/327
64/327
65/327
66/327
67/327
68/327
69/327
70/327
71/327
72/327
73/327
74/327
75/327
76/327
77/327
78/327
79/327
80/327
81/327
82/327
83/327
84/327
85/327
86/327
87/327
88/327
89/327
90/327
91/327
92/327
93/327
94/327
95/327
96/327
97/327
98/327
99/327
100/327
101/327
102/327
103/327
104/327
105/327
106/327
107/327
108/327
109/327
110/327
111/327
112/327
113/327
114/327
115/327
116/327
11

/Users/charlielezin/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['abaissement', 'abaisser', 'abandon', 'abandonne', 'abandonner', 'abandonné', 'abattage', 'abattement', 'abattoir', 'abattre']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Topic modeling


FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/Elements/JO_débats_1956-1958/débats_texte/model_ML/lda_model.pkl'

On récupère un DataFrame de 327 lignes (chaque fichier pdf transformé en fichier texte), avec trois colonnes : le nom du fichier, le texte entier, et la liste des mots du texte. 

In [27]:
df 

,date,text,bag_of_words
0,1956_i100,/ \n* Année 1956. — N° 100 A. N. ue Numéro ...,"[année, ue, numéro, franc, jeudi, octobre, jou..."
1,1956_i101,* AÂnée M. N° 101 A.^. Lie Numéro: 15 fra...,"[née, lie, numéro, franc, tt, mercredi, octobr..."
2,1956_i102,* Année 1956. N° 102 A. N. Le Numéro: 15 ...,"[année, numéro, franc, jeudi, octobre, journal..."
3,1956_i103,j \n* Année 1956. — N» 103 A. N. Le Numéro ...,"[année, numéro, franc, vendredi, octobre, jour..."
4,1956_i104,Y \ni i f l\ \n* Année 1958. — N° 104 A. N....,"[année, cy, numéro, franc, samedi, octobre, jo..."
...,...,...,...
322,1958_i60,% Année «88. — W 60 A. N. Le Numéro: 50 ...,"[année, numéro, franc, samedi, octobre, républ..."
323,1958_i62,",r* Année 1958. — N» 62 A. N. \n- i• Le Numé...","[année, numéro, franc, samedi, novembre, fe, j..."
324,1958_i7,Année 1958. — N» T A. N. Le Numéro: 50 fr...,"[année, numéro, franc, mercredi, janvier, jour..."
325,1958_i8,* Année 1958. - N° 8 A. N. Le Numéro: 50 f...,"[année, numéro, franc, jeudi, janvier, journal..."


In [54]:
df.to_csv(path_data + 'Mots_Débats.csv', index=False)

In [36]:
# Affiche la première ligne
print(df.iloc[0])

date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [37]:
cell = df.iloc[0, 2]  # première ligne, deuxième colonne
print(cell)

['année', 'ue', 'numéro', 'franc', 'jeudi', 'octobre', 'journal', 'officiel', 'république', 'fran', 'aise', 'débat', 'parlementaire', 'assemblee', 'nationale', 'compte', 'rendu', 'in', 'extenso', 'séance', 'question', 'écrites', 'reponses', 'ministre', 'question', 'abonnement', 'édition', 'débat', 'assemblée', 'nationale', 'aréststopolls', 'ex', 'frakce', 'xj', 'otjxzr', 'mei', 'fr', 'étranger', 'oo', 'fr', 'compte', 'chèque', 'postal', 'pari', 'pri', 're', 'joindre', 'derni', 're', 'bande', 'renouvellement', 'réclamation', 'direction', 'rédaction', 'administration', 'quai', 'voltaire', 'iv', 'pari', 'changement', 'adresse', 'ajouter', 'franc', 'session', 'legislature', 'compte', 'rendu', 'in', 'extenso', 'seance', 'séance', 'mercredi', 'octobre', 'sommaire', 'ipraeès', 'verbal', 'excuse', 'congé', 'désaccord', 'urgence', 'discussion', 'désignation', 'ipersonnes', 'contrainte', 'travail', 'pays', 'ennemi', 'suite', 'ja', 'discussion', 'lecture', 'proposition', 'loi', 'discussion', 'gén

In [55]:
mots_débats = df['bag_of_words']
mots_débats

0                                                                                                     [année, ue, numéro, franc, jeudi, octobre, journal, officiel, république, fran, aise, débat, parlementaire, assemblee, nationale, compte, rendu, in, extenso, séance, question, écrites, reponses, ministre, question, abonnement, édition, débat, assemblée, nationale, aréststopolls, ex, frakce, xj, otjxzr, mei, fr, étranger, oo, fr, compte, chèque, postal, pari, pri, re, joindre, derni, re, bande, renouvellement, réclamation, direction, rédaction, administration, quai, voltaire, iv, pari, changement, adresse, ajouter, franc, session, legislature, compte, rendu, in, extenso, seance, séance, mercredi, octobre, sommaire, ipraeès, verbal, excuse, congé, désaccord, urgence, discussion, désignation, ipersonnes, contrainte, travail, pays, ennemi, suite, ja, discussion, lecture, proposition, loi, discussion, générale, suite, mérigonde, chevigny, clôture, article, ...]
1                            